In [ ]:
from ultralytics import YOLO
import cv2
import numpy as np
from matplotlib import pyplot as plt
from PIL import Image, ImageDraw, ImageOps
import IPython
from utils import classname_to_color_mapper, draw_prediction_on_face, visualize_annotations



def imshow(img,size=(500,500)):
    _,ret = cv2.imencode('.jpg', img) 
    i = IPython.display.Image(data=ret,width =size[0],height =size[1])
    IPython.display.display(i)


model = YOLO('runs/segment/train_export_v6_large_augmented/weights/best.pt')  # load a pretrained model (recommended for training)


image_name = "cubemap_P74_5d1a4ffbcb8b43d55bb8af35519d6c41_negy"
#"cubemap_P74_5d1a4ffbcb8b43d55bb8af35519d6c41"
image_list = []

# Load your RGB image
image_base = Image.open(f'datasets/darwin_export_v6/images/test/{image_name}.png')


if image_name.endswith("posx"):
    pass
elif image_name.endswith("negx"):
    pass
elif image_name.endswith("negy"):
    #image_base = image_base.rotate(270)
    pass
elif image_name.endswith("posy"):
    #image_base = image_base.rotate(90)
    pass
elif image_name.endswith("posz"):
    pass
elif image_name.endswith("negz"):
    pass

image_1 = np.array(image_base)[..., 2::-1]
image_2 = np.array(image_base.rotate(90))[..., 2::-1]

image_list.append(image_1)
image_list.append(image_2)


result_of_the_segmentation = model(image_list,save=False)

face_prediction_list = [None]*len(result_of_the_segmentation)

for i in range(len(result_of_the_segmentation)):
    face_prediction_list[i] = draw_prediction_on_face(image_list[i],result_of_the_segmentation[i])

for i in range(len(result_of_the_segmentation)):
    imshow(image_list[i])
    imshow(face_prediction_list[i])
imshow(visualize_annotations(image_name[:-5]))
#plt.imshow(img,cmap='gray')
#plt.show()

In [ ]:
import os

FACES_LIST = ['_posx','_negx','_posy','_negy','_posz','_negz']



def create_full_image_from_faces(face_path_list):
    width,height = 1344,1344

    # if face_path_list is the name of a face or image
    if type(face_path_list) == str:
        if face_path_list[-5:] in FACES_LIST:
            img_name = face_path_list[:-5]
        else:
            img_name = face_path_list
        if os.path.isfile(f"SISTEMAS_UFF/Images/{img_name}_posx.png"):
            face_path_list = [img_name+FACES_LIST[i] for i in range(len(FACES_LIST))]
        else:
            raise SystemExit(f"image {img_name} not found in default folder 'SISTEMAS_UFF/Images/'")
            return 0

    complete_image = np.zeros((height*3,width*4,3))
    for face_name in face_path_list:
        image_original = Image.open(f"SISTEMAS_UFF/Images/{face_name}.png")
        
        if face_name.lower().endswith("posx"):
            pass
        elif face_name.lower().endswith("negx"):
            pass
        elif face_name.lower().endswith("negy"):
            image_original = image_original.rotate(270)
            pass
        elif face_name.lower().endswith("posy"):
            image_original = image_original.rotate(90)
            pass
        elif face_name.lower().endswith("posz"):
            pass
        elif face_name.lower().endswith("negz"):
            pass
        
        img = np.array(image_original) 
        height_pos = 0
        width_pos = 0
        if face_name.endswith("posx"):
            height_pos = 1
            width_pos = 2
            pass
        elif face_name.endswith("negx"):
            height_pos = 1
            width_pos = 0
            pass
        elif face_name.endswith("negy"):
            height_pos = 0
            width_pos = 1
            pass
        elif face_name.endswith("posy"):
            height_pos = 2
            width_pos = 1
            pass
        elif face_name.endswith("posz"):
            height_pos = 1
            width_pos = 1
            pass
        elif face_name.endswith("negz"):
            height_pos = 1
            width_pos = 3
            pass
        complete_image[height*height_pos:height*(height_pos+1),width*width_pos:width*(width_pos+1),:] = img
    complete_image[..., :3] = complete_image[..., 2::-1]
    return complete_image


def extract_face_from_cubemap(cubemap):
    width,height = 1344,1344

    topo     = np.s_[height*0:height*1,width*1:width*2]
    esquerda = np.s_[height*1:height*2,width*0:width*1]
    centro   = np.s_[height*1:height*2,width*1:width*2]
    direita  = np.s_[height*1:height*2,width*2:width*3]
    contrario  = np.s_[height*1:height*2,width*3:width*4]
    base     = np.s_[height*2:height*3,width*1:width*2]

    lista_faces = [esquerda,centro,direita,contrario]

    face_1 = cubemap[esquerda]
    face_2 = cubemap[centro]
    face_3 = cubemap[direita]
    face_4 = cubemap[contrario]

    faces = [face_1,face_2,face_3,face_4]

    face_topo = cubemap[topo]
    face_base = cubemap[base]
    
    canvas = np.zeros((height*3,width*3,3))
    face_list = []

    for face_number in range(len(faces)):
        canvas[topo]     = np.rot90(face_topo, k=face_number, axes=(1,0))
        canvas[esquerda] = np.rot90(faces[face_number], k=0, axes=(1,0))
        canvas[centro]   = np.rot90(faces[(face_number+1)%4], k=0, axes=(1,0))
        canvas[direita]  = np.rot90(faces[(face_number+2)%4], k=0, axes=(1,0))
        canvas[base]     = np.rot90(face_base, k=-face_number, axes=(1,0))
        face_list.append(np.copy(canvas))
    
    canvas[topo]     = np.rot90(faces[3], k=2, axes=(1,0))
    canvas[esquerda] = np.rot90(faces[0], k=1, axes=(1,0))
    canvas[centro]   = np.rot90(face_topo, k=0, axes=(1,0))
    canvas[direita]  = np.rot90(faces[2], k=-1, axes=(1,0))
    canvas[base]     = np.rot90(faces[1], k=-0, axes=(1,0))
    face_list.append(np.copy(canvas))

    canvas[topo]     = np.rot90(faces[1], k=0, axes=(1,0))
    canvas[esquerda] = np.rot90(faces[0], k=-1, axes=(1,0))
    canvas[centro]   = np.rot90(face_base, k=0, axes=(1,0))
    canvas[direita]  = np.rot90(faces[2], k=1, axes=(1,0))
    canvas[base]     = np.rot90(faces[3], k=2, axes=(1,0))
    face_list.append(np.copy(canvas))

    return face_list


complete_image = create_full_image_from_faces(image_name)
imshow(complete_image)

face_list = extract_face_from_cubemap(complete_image)
        
width,height = 1344,1344

left_padding = 50
right_padding = 50

bot_padding = 50
top_padding = 50


padding_index = np.s_[height*1-left_padding:height*2+right_padding,width*1-bot_padding:width*2+top_padding]
    
central_face = np.s_[left_padding:height+right_padding,bot_padding:width+top_padding]


topo     = np.s_[height*0:height*1,width*1:width*2]
esquerda = np.s_[height*1:height*2,width*0:width*1]
centro   = np.s_[height*1:height*2,width*1:width*2]
direita  = np.s_[height*1:height*2,width*2:width*3]
contrario  = np.s_[height*1:height*2,width*3:width*4]
base     = np.s_[height*2:height*3,width*1:width*2]

face_position_list = [centro,direita,contrario,esquerda,topo,base]

canvas = np.zeros((height,width,4))
canvas_rotated_list = [np.zeros((height,width,4)) for _ in range(3)]

face = face_list[4]
face_cutted = face[padding_index]
#imshow(face_cutted)


result_of_the_segmentation = model(face_cutted,save=False, verbose=False)
mask = draw_prediction_on_face(face_cutted[..., 2::-1],result_of_the_segmentation[0])
#imshow(mask[central_face])
canvas = mask[central_face]

for i_1 in range(3):
    canvas_rotated = canvas_rotated_list[i_1]
    result_of_the_segmentation_rotated = model(np.rot90(face_cutted,i_1+1),save=False, verbose=False)
    mask_rotated = draw_prediction_on_face(np.rot90(face_cutted[..., 2::-1],i_1+1),result_of_the_segmentation_rotated[0])
    canvas_rotated_list[i_1] = np.rot90(mask_rotated[central_face],-(i_1+1))


imshow(canvas,(1000,1000))

for i_1 in range(3):
    imshow(canvas_rotated_list[i_1],(1000,1000))


In [ ]:
paddings_values_list = [0,50]

paddings_values_list_len = len(paddings_values_list)

canvas_with_different_paddings_complete = np.zeros((height*(paddings_values_list_len**2),width*(paddings_values_list_len**2),4))

for i_0 in range(paddings_values_list_len):
    left_padding = paddings_values_list[i_0]

    for i_1 in range(paddings_values_list_len):
        right_padding = paddings_values_list[i_1]

        for i_2 in range(paddings_values_list_len):
            bot_padding = paddings_values_list[i_2]

            for i_3 in range(paddings_values_list_len):
                top_padding = paddings_values_list[i_3]


                padding_index = np.s_[height*1-left_padding:height*2+right_padding,width*1-bot_padding:width*2+top_padding]
                    
                central_face = np.s_[left_padding : left_padding+height, 
                                     bot_padding  : bot_padding+width]

                #canvas_with_different_paddings = np.zeros((height,width,4))
                #canvas_rotated_list = [np.zeros((height,width,4)) for _ in range(3)]

                face = face_list[3]
                face_cutted = face[padding_index]
                #print(face_cutted.shape)
                #imshow(face_cutted)


                result_of_the_segmentation = model(face_cutted,save=False, verbose=False)
                mask_with_different_paddings = draw_prediction_on_face(face_cutted[..., 2::-1],result_of_the_segmentation[0])
                #imshow(mask[central_face])
                #print(mask_with_different_paddings.shape)
                #print(central_face)
                canvas_with_different_paddings = mask_with_different_paddings[central_face]
                #print(canvas_with_different_paddings.shape)

                """
                print(f"Top padding: {top_padding}")
                print(f"Bot padding: {bot_padding}")
                print(f"Right padding: {right_padding}")
                print(f"Left padding: {left_padding}")
                #imshow(canvas_with_different_paddings,(1000,1000))
                #"""

                x_index = i_3 + (i_2*paddings_values_list_len)
                y_index = i_1 + (i_0*paddings_values_list_len)

                #print(canvas_with_different_paddings.shape)
                canvas_with_different_paddings_complete[x_index*height:x_index*height+height,y_index*width:y_index*width+width] = canvas_with_different_paddings
                #print("-------------------------------------------")

imshow(canvas_with_different_paddings_complete)

In [ ]:

imshow(visualize_annotations(image_name[:-5],dont_show_ground_truth=True)[topo])

In [ ]:
from PIL import Image, ImageDraw, ImageOps
from ultralytics import YOLO
import cv2
import numpy as np
import IPython

def imshow(img,size=(500,500)):
    _,ret = cv2.imencode('.jpg', img) 
    i = IPython.display.Image(data=ret,width =size[0],height =size[1])
    IPython.display.display(i)

# Load a model
#model = YOLO("yolov8n.yaml")  # build a new model from scratch
model_original = YOLO("yolov8l-seg.pt")  # load a pretrained model (recommended for training)

# Use the model
#model.train(data="coco128.yaml", epochs=3)  # train the model
#metrics = model.val()  # evaluate model performance on the validation set

image_name_list = [ "000000005238.jpg",
                    "000000005134.jpg",
                    "000000005148.jpg",
                    "000000001097.jpg",
                    "000000007127.jpg",
                    "000000006321.jpg",]

img_trasnformed_list = []
for image_name in image_name_list:
    image_base = Image.open(f'datasets/coco/images/test2017/{image_name}')

    img_trasnformed_list.append(image_base.rotate(0))
    img_trasnformed_list.append(image_base.rotate(90))
    img_trasnformed_list.append(image_base.rotate(180))
    img_trasnformed_list.append(image_base.rotate(270))
    img_trasnformed_list.append(ImageOps.mirror(image_base))

    w,h = image_base.size
    #img_trasnformed_list.append(image_base.crop((0,           0,          w,          int(h*0.9))))
    img_trasnformed_list.append(image_base.crop((0,           0,          int(w*0.9), h)))
    img_trasnformed_list.append(image_base.crop((int(w*0.1),  0,          w,          h)))
    #img_trasnformed_list.append(image_base.crop((0,           int(h*0.1), w,          h)))

model_original(img_trasnformed_list, save=True,name="testing_consistency", exist_ok=True,device=[1])  # predict on an image
    

#
#image_bus_reflected = cv2.copyMakeBorder(np.array(Image.open(f'bus.jpg'))[:, :, ::-1].copy(),100,100,100,100,cv2.BORDER_REFLECT_101)
#
#
#img_bus_list = [image_bus,image_bus_rotated_90,image_bus_rotated_180,image_bus_rotated_270,image_bus_flipped, image_bus_resized,image_bus_reflected]
#
#results = model_original(img_bus_list, save=True,name="testing_consistency", exist_ok=True)  # predict on an image
#results[0].show()
#path = model.export(format="onnx")  # export the model to ONNX format